In [1]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#ccba
ccba = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_ccba_full_hashed.csv')
#cdtx
cdtx = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_cdtx0001_full_hashed.csv')
#custinfo
custinfo = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_custinfo_full_hashed.csv')
#dp
dp = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_dp_full_hashed.csv')
#remit
remit = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_remit1_full_hashed.csv')
#train_alert_time
train_alert_time = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/train_x_alert_date.csv')
#predict_alert_time
predict_alert_time = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_x_alert_date.csv')
#y
y = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/train_y_answer.csv')
#案件名單
doc = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/預測的案件名單及提交檔案範例.csv')
#prev_list set to 0 before submission
doc_merged = custinfo.merge(doc, on='alert_key', how='right').merge(predict_alert_time, on='alert_key', how='left')
doc_merged = doc_merged[~doc_merged['cust_id'].isnull()]
max_doc = doc_merged.groupby('cust_id')['date'].max().reset_index()
max_doc.rename(columns={'date':'max_date'},inplace=True)
doc_merged = doc_merged.merge(max_doc, on='cust_id', how='left')
prev_list = doc_merged[doc_merged['max_date']>doc_merged['date']]['alert_key'].tolist()

In [3]:
def train_alert_process_func(data, custinfo, predict_alert_time):
  alert_data = data[data['cust_id'].isin(custinfo[custinfo['alert_key'].isin(predict_alert_time['alert_key'].tolist())]['cust_id'].tolist())]
  train_data = data[~data['cust_id'].isin(custinfo[custinfo['alert_key'].isin(predict_alert_time['alert_key'].tolist())]['cust_id'].tolist())]
  train_data['y'] = 0
  sar_idx = train_data[train_data['cust_id'].isin(custinfo[custinfo['alert_key'].isin(y[y['sar_flag']==1]['alert_key'].tolist())]['cust_id'].tolist())].index
  train_data.loc[sar_idx, 'y'] = 1
  return train_data, alert_data

In [4]:
#Data Cleansing
#SAR戶僅留前X天
def train_prev_d(x, day):
  prev_d = x.groupby('cust_id')['tx_date'].max() - day
  prev_d = prev_d.reset_index()
  prev_d.rename(columns={'tx_date':'prev_d'}, inplace=True)
  x = x.merge(prev_d, on='cust_id', how='left')
  x.drop(x[x['tx_date']<x['prev_d']].index,inplace=True)
  x.pop('prev_d')
  return x
#特徵前處理
def preprocess(data):
  dict1 = {}
  idx = 0
  num = 0
  for i in range(0,395,1):
    dict1[i] = str(idx)
    num += 1
    if num == 7:
      idx += 1
      num = 0
  data['tx_date_group'] = data.tx_date.map(lambda x: dict1[x])
  data['session_cust_id'] = data.tx_date_group + data.cust_id
  data['date_cust_id'] = data.tx_date.astype(str) + data.cust_id
  data['date_time_cust_id'] = data.tx_date.astype(str) + data.tx_date.astype(str) + data.cust_id
  data['tx_year'] = 2021
  data['tx_date_formal'] = data['tx_date']+1
  data.loc[data[data['tx_date_formal']>365].index,'tx_year'] = 2022
  data.loc[data[data['tx_date_formal']>365].index, 'tx_date_formal'] = \
  data.loc[data[data['tx_date_formal']>365].index, 'tx_date_formal'] -365
  data["tx_date_formal"] = data["tx_year"]*1000 + data["tx_date_formal"]
  data['tx_date_formal'] = pd.to_datetime(data['tx_date_formal'], format="%Y%j")
  return data

In [5]:
def cdtx_feature_func(data):
  ##(t-1)d diff
  #data['prev_d_diff'] = data['trans_date'] - data.groupby('cust_id')['trans_date'].shift(1)
  #data['prev_d_diff'].fillna(15, inplace=True)
  #amt
  all_amt_sum = (data.groupby('cust_id')['amt'].sum()/data.groupby('cust_id')['amt'].sum().max())\
              .rename('all_amt_sum').reset_index()
  data = data.merge(all_amt_sum, on='cust_id', how='left')
  date_amt_sum = (data.groupby(['cust_id','date'])['amt'].sum()/data.groupby(['cust_id','date'])['amt'].sum().max())\
              .rename('date_amt_sum').reset_index()
  data = data.merge(date_amt_sum, on=['cust_id','date'], how='left')
  #txn_cnt
  all_txn_cnt = data.groupby('cust_id')['date'].count().rename('cdtx_all_txn_cnt').reset_index()
  data = data.merge(all_txn_cnt, on='cust_id', how='left')
  data['cdtx_all_txn_cnt'].fillna(0, inplace=True)
  date_txn_cnt = data.groupby(['cust_id','date'])['amt'].count().rename('date_txn_cnt').reset_index()
  data = data.merge(date_txn_cnt, on=['cust_id','date'], how='left')
  #country
  all_country_cnt = data.groupby('cust_id')['country'].count().rename('cdtx_all_country_cnt').reset_index()
  data = data.merge(all_country_cnt, on='cust_id', how='left')
  data['cdtx_all_country_cnt'].fillna(0, inplace=True)
  date_country_cnt = data.groupby(['cust_id','date'])['country'].count().rename('date_country_cnt').reset_index()
  data = data.merge(date_country_cnt, on=['cust_id','date'], how='left')
  #cur_type
  all_cur_type_cnt = data.groupby('cust_id')['cur_type'].count().rename('cdtx_all_cur_type_cnt').reset_index()
  data = data.merge(all_cur_type_cnt, on='cust_id', how='left')
  data['cdtx_all_cur_type_cnt'].fillna(0, inplace=True)
  date_cur_type_cnt = data.groupby(['cust_id','date'])['cur_type'].count().rename('date_cur_type_cnt').reset_index()
  data = data.merge(date_cur_type_cnt, on=['cust_id','date'], how='left')
  #max_min_date_diff
  max_min_date_diff = data.groupby(['cust_id']).apply(lambda x: x['date'].max() - x['date'].min()).rename('max_min_date_diff').reset_index()
  data = data.merge(max_min_date_diff, on='cust_id', how='left')
  return data

In [6]:
def remit_feature_func(data):
  ##(t-1)d diff
  #data['prev_d_diff'] = data['trans_date'] - data.groupby('cust_id')['trans_date'].shift(1)
  #data['prev_d_diff'].fillna(15, inplace=True)
  #amt
  all_amt_sum = (data.groupby('cust_id')['trade_amount_usd'].sum()/data.groupby('cust_id')['trade_amount_usd'].sum().max())\
              .rename('all_amt_sum').reset_index()
  data = data.merge(all_amt_sum, on='cust_id', how='left')
  time_amt_sum = (data.groupby(['cust_id','trans_date'])['trade_amount_usd'].sum()/data.groupby(['cust_id','trans_date'])['trade_amount_usd'].sum().max())\
              .rename('time_amt_sum').reset_index()
  data = data.merge(time_amt_sum, on=['cust_id','trans_date'], how='left')
  data['trans_date'] = data['trans_date']/data['trans_date'].max()
  #txn_cnt
  all_txn_cnt = data.groupby('cust_id')['trans_date'].count().rename('remit_all_txn_cnt').reset_index()
  data = data.merge(all_txn_cnt, on='cust_id', how='left')
  data['remit_all_txn_cnt'].fillna(0, inplace=True)
  date_txn_cnt = data.groupby(['cust_id','trans_date'])['trans_no'].count().rename('date_txn_cnt').reset_index()
  data = data.merge(date_txn_cnt, on=['cust_id','trans_date'], how='left')
  #max_min_date_diff
  max_min_date_diff = data.groupby(['cust_id']).apply(lambda x: x['trans_date'].max() - x['trans_date'].min()).rename('max_min_date_diff').reset_index()
  data = data.merge(max_min_date_diff, on='cust_id', how='left')
  remit_pivot = data.groupby(['cust_id','trans_date','trans_no'])['trade_amount_usd'].sum().reset_index().pivot_table(index=['cust_id','trans_date'],columns='trans_no',values='trade_amount_usd').reset_index()
  remit_pivot.fillna(0, inplace=True)
  remit_col = list(remit_pivot.columns)
  remit_col.remove('cust_id')
  remit_col.remove('trans_date')
  remit_pivot['remit_txn_sum'] = remit_pivot[remit_col].sum(axis=1)
  for col in remit_col:
    remit_pivot[col] = remit_pivot[col]/remit_pivot['remit_txn_sum']
  for col in list(set([num for num in range(0,5,1)]) -  set(remit_pivot.columns[2:])):
    remit_pivot[col] = 0.0
  remit_pivot.rename(columns={0:'remit_trans_no_0',1:'remit_trans_no_1',2:'remit_trans_no_2',3:'remit_trans_no_3',4:'remit_trans_no_4'}, inplace=True)
  remit_pivot.fillna(0, inplace=True)
  data = data.merge(remit_pivot, on=['cust_id', 'trans_date'], how='left')
  return data

In [7]:
#session 交易差額比率
def dp_feature_func(data):
  session_amt_diff = data.groupby(['session_cust_id','debit_credit'])['tx_amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='session_cust_id', columns='debit_credit', values='tx_amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['session_amt_diff_ratio'] = \
    abs(session_amt_diff['CR'] - session_amt_diff['DB']) / abs(session_amt_diff['CR'] + session_amt_diff['DB'])
  session_amt_diff = session_amt_diff.reset_index()[['session_cust_id','session_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='session_cust_id', how='left')
  #當日 交易差額比率
  date_amt_diff = data.groupby(['date_cust_id','debit_credit'])['tx_amt'].sum().reset_index()
  date_amt_diff = pd.pivot_table(date_amt_diff, index='date_cust_id', columns='debit_credit', values='tx_amt')
  date_amt_diff.fillna(1, inplace=True)
  date_amt_diff['date_amt_diff_ratio'] = \
  abs(date_amt_diff['CR'] - date_amt_diff['DB']) / abs(date_amt_diff['CR'] + date_amt_diff['DB'])
  date_amt_diff = date_amt_diff.reset_index()[['date_cust_id','date_amt_diff_ratio']]
  data = data.merge(date_amt_diff, on=['date_cust_id'], how='left')
  #當時 交易差額比率
  date_time_amt_diff = data.groupby(['date_time_cust_id','debit_credit'])['tx_amt'].sum().reset_index()
  date_time_amt_diff = pd.pivot_table(date_time_amt_diff, index='date_time_cust_id', columns='debit_credit', values='tx_amt')
  date_time_amt_diff.fillna(1, inplace=True)
  date_time_amt_diff['date_time_amt_diff_ratio'] = \
  abs(date_time_amt_diff['CR'] - date_time_amt_diff['DB']) / abs(date_time_amt_diff['CR'] + date_time_amt_diff['DB'])
  date_time_amt_diff = date_time_amt_diff.reset_index()[['date_time_cust_id','date_time_amt_diff_ratio']]
  data = data.merge(date_time_amt_diff, on=['date_time_cust_id'], how='left')
  #當時交易筆數 tx_cnt_date_time
  tx_cnt_date_time = data.groupby(['cust_id','tx_date','tx_time'])['debit_credit'].count().reset_index()
  tx_cnt_date_time.rename(columns={'debit_credit':'tx_cnt_date_time'}, inplace=True)
  data = data.merge(tx_cnt_date_time, on=['cust_id','tx_date','tx_time'], how='left')
  #當日交易筆數 tx_cnt_date
  tx_cnt_date = data.groupby(['cust_id','tx_date'])['debit_credit'].count().reset_index()
  tx_cnt_date.rename(columns={'debit_credit':'tx_cnt_date'}, inplace=True)
  data = data.merge(tx_cnt_date, on=['cust_id','tx_date'], how='left')
  #當時總分行數 txbranch_day_cnt
  txbranch_day_time_cnt = data.groupby(['cust_id','tx_date','tx_time'])['txbranch'].count().reset_index()
  txbranch_day_time_cnt.rename(columns={'txbranch':'txbranch_day_time_cnt'}, inplace=True)
  data = data.merge(txbranch_day_time_cnt, on=['cust_id','tx_date','tx_time'], how='left')
  #單日總分行數 txbranch_day_cnt
  txbranch_day_cnt = data.groupby(['cust_id','tx_date'])['txbranch'].count().reset_index()
  txbranch_day_cnt.rename(columns={'txbranch':'txbranch_day_cnt'}, inplace=True)
  data = data.merge(txbranch_day_cnt, on=['cust_id','tx_date'], how='left')
  #當日ATM 佔交易數比例
  day_atm_txn_ratio = data.groupby(['cust_id','tx_date'])['ATM'].sum().reset_index()
  day_atm_txn_ratio.rename(columns={'ATM':'day_atm_txn_ratio'}, inplace=True)
  data = data.merge(day_atm_txn_ratio, on=['cust_id','tx_date'], how='left')
  data.day_atm_txn_ratio = data.day_atm_txn_ratio / data.tx_cnt_date
  #當時ATM 佔交易數比例
  day_time_atm_txn_ratio = data.groupby(['cust_id','tx_date','tx_time'])['ATM'].sum().reset_index()
  day_time_atm_txn_ratio.rename(columns={'ATM':'day_time_atm_txn_ratio'}, inplace=True)
  data = data.merge(day_time_atm_txn_ratio, on=['cust_id','tx_date','tx_time'], how='left')
  data.day_time_atm_txn_ratio = data.day_time_atm_txn_ratio / data.tx_cnt_date_time
  #當日跨行 佔交易數比例
  day_cross_bank_ratio = data.groupby(['cust_id','tx_date'])['cross_bank'].sum().reset_index()
  day_cross_bank_ratio.rename(columns={'cross_bank':'day_cross_bank_ratio'}, inplace=True)
  data = data.merge(day_cross_bank_ratio, on=['cust_id','tx_date'], how='left')
  data.day_cross_bank_ratio = data.day_cross_bank_ratio / data.tx_cnt_date
  #當時跨行 佔交易數比例
  day_time_cross_bank_ratio = data.groupby(['cust_id','tx_date','tx_time'])['cross_bank'].sum().reset_index()
  day_time_cross_bank_ratio.rename(columns={'cross_bank':'day_time_cross_bank_ratio'}, inplace=True)
  data = data.merge(day_time_cross_bank_ratio, on=['cust_id','tx_date','tx_time'], how='left')
  data.day_time_cross_bank_ratio = data.day_time_cross_bank_ratio / data.tx_cnt_date_time

  ##
  #當session交易筆數 tx_cnt_session
  tx_cnt_session = data.groupby(['session_cust_id'])['debit_credit'].count().reset_index()
  tx_cnt_session.rename(columns={'debit_credit':'tx_cnt_session'}, inplace=True)
  data = data.merge(tx_cnt_session, on=['session_cust_id'], how='left')
  #當session總分行數 txbranch_session_cnt
  txbranch_session_cnt = data.groupby(['session_cust_id'])['txbranch'].count().reset_index()
  txbranch_session_cnt.rename(columns={'txbranch':'txbranch_session_cnt'}, inplace=True)
  data = data.merge(txbranch_session_cnt, on=['session_cust_id'], how='left')
  #當session跨行 佔交易數比例
  session_cross_bank_ratio = data.groupby(['session_cust_id'])['cross_bank'].sum().reset_index()
  session_cross_bank_ratio.rename(columns={'cross_bank':'session_cross_bank_ratio'}, inplace=True)
  data = data.merge(session_cross_bank_ratio, on=['session_cust_id'], how='left')
  data.session_cross_bank_ratio = data.session_cross_bank_ratio / data.tx_cnt_date
  #當sessionATM 佔交易數比例
  session_atm_txn_ratio = data.groupby(['session_cust_id'])['ATM'].sum().reset_index()
  session_atm_txn_ratio.rename(columns={'ATM':'session_atm_txn_ratio'}, inplace=True)
  data = data.merge(session_atm_txn_ratio, on=['session_cust_id'], how='left')
  data.session_atm_txn_ratio = data.session_atm_txn_ratio / data.tx_cnt_date
  #time_diff
  distinct_date_time = data[['cust_id','tx_date','tx_time']].drop_duplicates().sort_values(['cust_id','tx_date','tx_time']).reset_index(drop=True)
  distinct_date_time['date_diff'] = distinct_date_time.groupby('cust_id').apply(lambda x: x['tx_date'] - x['tx_date'].shift(1)).reset_index(drop=True)
  distinct_date_time['time_diff'] = distinct_date_time.groupby('cust_id').apply(lambda x: x['tx_time'] - x['tx_time'].shift(1)).reset_index(drop=True)
  distinct_date_time.fillna(0, inplace=True)
  distinct_date_time['time_diff'] = (distinct_date_time['date_diff']*24) + distinct_date_time['time_diff']
  data = data.merge(distinct_date_time[['cust_id', 'tx_date', 'tx_time', 'time_diff']], on=['cust_id', 'tx_date', 'tx_time'], how='left')

  return data

In [8]:
def model_training_1(data, columns):
  #逐筆交易處理
  Y = data['y']
  X = data[columns[1:-1]]
  test_size = 0.2
  seed = 42
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)#, stratify = Y)
  model = XGBClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  predictions = [round(value) for value in y_pred]
  # evaluate predictions
  accuracy = accuracy_score(y_test, predictions)
  print("Accuracy: %.2f%%" % (accuracy * 100.0))
  f1 = f1_score(y_test, predictions)
  print("F1 Scoure: %.2f%%" % (f1 * 100.0))
  print(precision_score(y_test, predictions))
  print(recall_score(y_test, predictions))
  feature_importance = pd.DataFrame({'columns':list(X.columns),'score':model.feature_importances_})
  return X, model, feature_importance

In [9]:
#彙整model training 1偵測結果
#predict_proba以0.05機率區間為一個欄位判斷分佈
def mapping(proba):
    if proba <= 0.1:
        return 1
    elif 0.1 < proba <= 0.2:
        return 2
    elif 0.2 < proba <= 0.3:
        return 3
    elif 0.3 < proba <= 0.4:
        return 4
    elif 0.4 < proba <= 0.5:
        return 5
    elif 0.5 < proba <= 0.6:
        return 6
    elif 0.6 < proba <= 0.7:
        return 7
    elif 0.7 < proba <= 0.8:
        return 8
    elif 0.8 < proba <= 0.9:
        return 9
    elif 0.9 < proba <= 1:
        return 10
def debit_credit_ratio_func(data):
  #id
  debit_credit_ratio = data.groupby(['cust_id'])['debit_credit'].value_counts().rename('debit_credit_ratio').reset_index()
  debit_credit_ratio = debit_credit_ratio.pivot_table(values='debit_credit_ratio', index=['cust_id'], columns='debit_credit')
  debit_credit_ratio.fillna(0, inplace=True)
  debit_credit_ratio['debit_credit_ratio'] = debit_credit_ratio['DB']/debit_credit_ratio.sum(axis=1)
  debit_credit_ratio = debit_credit_ratio.reset_index()[['cust_id','debit_credit_ratio']]
  data = data.merge(debit_credit_ratio, on=['cust_id'], how='left')
  return debit_credit_ratio
def all_txn_cnt(data):
  all_txn_cnt = data.groupby('cust_id')['tx_date'].count().rename('all_txn_cnt').reset_index()
  data = data.merge(all_txn_cnt, on='cust_id', how='left')
  return all_txn_cnt
def result_preprocess_func(data, X, model):
  data_db_cr_ratio = debit_credit_ratio_func(data)
  data_all_txn_cnt = all_txn_cnt(data)
  data['proba'] = model.predict_proba(X)[:,1]
  result = data[['cust_id','proba','y']]
  result['level'] = result["proba"].map(mapping)
  result = result[['cust_id', 'level', 'y']]
  result = result.groupby(['cust_id','level']).count().reset_index().pivot_table(index='cust_id', columns='level', values='y')
  result.fillna(0, inplace=True)
  result = result.div(result.sum(axis=1), axis=0).reset_index()
  for col in list(set([num for num in range(1,11,1)]) -  set(result.columns[1:])):
    result[col] = 0.0
  result = result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
  result = result.merge(data[['cust_id','y']].drop_duplicates(), on='cust_id', how='left')
  result = result.merge(data_db_cr_ratio, on='cust_id', how='left')
  result = result.merge(data_all_txn_cnt, on='cust_id', how='left')
  return result

In [10]:
#model training 2
#歸戶判斷是否報SAR
def model_training_2(result):
  result_col = list(result.columns)
  result_col.remove('cust_id')
  result_col.remove('y')
  test_size = 0.2
  seed = 42
  X_train, X_test, y_train, y_test = train_test_split(result[result_col], result['y'], test_size=test_size, random_state=seed)#, stratify = result['y'])
  model = XGBClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  predictions = [round(value) for value in y_pred]
  # evaluate predictions
  accuracy = accuracy_score(y_test, predictions)
  print("Accuracy: %.2f%%" % (accuracy * 100.0))
  f1 = f1_score(y_test, predictions)
  print("F1 Scoure: %.2f%%" % (f1 * 100.0))
  print(precision_score(y_test, predictions))
  print(recall_score(y_test, predictions))
  return model, result_col

In [11]:
def alert_output(alert, model_1, model_2, alert_col, result_col, doc):
  data_db_cr_ratio = debit_credit_ratio_func(alert)
  data_all_txn_cnt = all_txn_cnt(alert)

  #Remit Submission
  alert_data = alert
  alert_data['proba'] = model_1.predict_proba(alert[alert_col])[:,1]
  alert_result = alert_data[['cust_id','proba']]
  alert_result['level'] = alert_result["proba"].map(mapping)
  alert_result = alert_result[['cust_id', 'level']]
  alert_result['cnt'] = 1
  alert_result = alert_result.groupby(['cust_id','level'])['cnt'].count().reset_index().pivot_table(index='cust_id', columns='level', values='cnt')
  alert_result.fillna(0, inplace=True)
  alert_result = alert_result.div(alert_result.sum(axis=1), axis=0)
  alert_result = alert_result.reset_index()
  for col in list(set([num for num in range(1,11,1)]) -  set(alert_result.columns[1:])):
    alert_result[col] = 0.0
  alert_result = alert_result.merge(data_db_cr_ratio, on='cust_id', how='left')
  alert_result = alert_result.merge(data_all_txn_cnt, on='cust_id', how='left')
  alert_pred = model_2.predict_proba(alert_result[result_col])
  # evaluate predictions
  alert_result = alert_result[['cust_id']]
  alert_result['probability'] = alert_pred[:,1]

  final = predict_alert_time.merge(custinfo[['alert_key', 'cust_id']].merge(alert_result, on='cust_id'), on='alert_key', how='left')[['alert_key', 'probability']]
  doc = doc[['alert_key']]
  final = doc.merge(final, on='alert_key', how='left')
  final.fillna(0,inplace=True)
  final.loc[final[final['alert_key'].isin(prev_list)].index,'probability']=0
  return final, alert_result

In [12]:
def prev_7d_feature_func(data):
  #last 7 days processing
  data_distinct = data[['cust_id', 'tx_date_formal','tx_date']].drop_duplicates().reset_index(drop=True)
  cross_bank_sum = data.groupby(['cust_id','tx_date_formal'])['cross_bank'].sum().rename('cross_bank_sum').reset_index()
  prev_7d_data = data_distinct.merge(cross_bank_sum, on=['cust_id','tx_date_formal'], how='left')
  dbcr_amt_sum = data.groupby(['cust_id','tx_date_formal','debit_credit'])['tx_amt'].sum().rename('debit_credit_sum').reset_index()
  dbcr_amt_sum = pd.pivot_table(dbcr_amt_sum, index=['cust_id','tx_date_formal'], columns='debit_credit', values='debit_credit_sum')
  prev_7d_data = prev_7d_data.merge(dbcr_amt_sum, on=['cust_id','tx_date_formal'], how='left')
  tx_date_sum = data.groupby(['cust_id','tx_date_formal'])['tx_amt'].sum().rename('tx_amt_sum')
  prev_7d_data = prev_7d_data.merge(tx_date_sum, on=['cust_id','tx_date_formal'], how='left')
  tx_cnt = data.groupby(['cust_id','tx_date_formal'])['tx_amt'].count().rename('tx_cnt').reset_index()
  prev_7d_data = prev_7d_data.merge(tx_cnt, on=['cust_id','tx_date_formal'], how='left')
  #當日ATM 佔交易數比例
  day_atm_cnt = data.groupby(['cust_id','tx_date_formal'])['ATM'].sum().rename('day_atm_cnt').reset_index()
  prev_7d_data = prev_7d_data.merge(day_atm_cnt, on=['cust_id','tx_date_formal'], how='left')
  txbranch_day_cnt = data.groupby(['cust_id','tx_date_formal'])['txbranch'].count().rename('txbranch_day_cnt').reset_index()
  prev_7d_data = prev_7d_data.merge(txbranch_day_cnt, on=['cust_id','tx_date_formal'], how='left')
  prev_7d_data.fillna(0, inplace=True)
  prev_7d_data = prev_7d_data.sort_values(['cust_id','tx_date_formal'])
  prev_7d_data = prev_7d_data.set_index('tx_date_formal')
  #feature engineering
  prev_7d_cross_bank_sum = prev_7d_data.groupby('cust_id')['cross_bank_sum'].rolling(window='7D').sum().rename('prev_7d_cross_bank_sum').reset_index()
  prev_7d_CR_sum = prev_7d_data.groupby('cust_id')['CR'].rolling(window='7D').sum().rename('prev_7d_CR_sum').reset_index()
  prev_7d_DB_sum = prev_7d_data.groupby('cust_id')['DB'].rolling(window='7D').sum().rename('prev_7d_DB_sum').reset_index()
  prev_7d_tx_cnt = prev_7d_data.groupby('cust_id')['tx_cnt'].rolling(window='7D').sum().rename('prev_7d_tx_cnt').reset_index()
  prev_7d_txbranch_cnt = prev_7d_data.groupby('cust_id')['txbranch_day_cnt'].rolling(window='7D').sum().rename('prev_7d_txbranch_cnt').reset_index()
  prev_7d_atm_cnt = prev_7d_data.groupby('cust_id')['day_atm_cnt'].rolling(window='7D').sum().rename('prev_7d_atm_cnt').reset_index()

  data_distinct = data_distinct.merge(prev_7d_CR_sum, on=['cust_id','tx_date_formal']).merge(prev_7d_DB_sum, on=['cust_id','tx_date_formal'])\
    .merge(prev_7d_tx_cnt, on=['cust_id','tx_date_formal']).merge(prev_7d_txbranch_cnt, on=['cust_id','tx_date_formal']).merge(prev_7d_atm_cnt, on=['cust_id','tx_date_formal'])\
    .merge(prev_7d_cross_bank_sum, on=['cust_id','tx_date_formal'])

  data_distinct['prev_7d_txbranch_ratio'] = data_distinct.prev_7d_txbranch_cnt / data_distinct.prev_7d_tx_cnt
  data_distinct['prev_7d_atm_ratio'] = data_distinct.prev_7d_atm_cnt / data_distinct.prev_7d_tx_cnt
  data_distinct['prev_7d_cross_bank_ratio'] = data_distinct.prev_7d_cross_bank_sum / data_distinct.prev_7d_tx_cnt
  data_distinct['prev7d_amt_diff_ratio'] = \
    abs(data_distinct['prev_7d_CR_sum'] - data_distinct['prev_7d_DB_sum']) / abs(data_distinct['prev_7d_CR_sum'] + data_distinct['prev_7d_DB_sum'])
  data_distinct = data_distinct[['cust_id','tx_date','prev_7d_tx_cnt','prev7d_amt_diff_ratio','prev_7d_txbranch_ratio','prev_7d_atm_cnt','prev_7d_atm_ratio',
                                 'prev_7d_txbranch_cnt','prev_7d_cross_bank_ratio','prev_7d_cross_bank_sum']]
  data = data.merge(data_distinct, on=['cust_id', 'tx_date'], how='left')
  return data, data_distinct

In [33]:
#dp zone
train_dp, alert_dp = train_alert_process_func(dp, custinfo, predict_alert_time)
train_dp = train_prev_d(train_dp, 30)
alert_dp = train_prev_d(alert_dp, 30)
train_dp = preprocess(train_dp)
alert_dp = preprocess(alert_dp)
##

train_dp, train_dp_distinct = prev_7d_feature_func(train_dp)
alert_dp, alert_dp_distinct = prev_7d_feature_func(alert_dp)
##
train_dp = dp_feature_func(train_dp)
alert_dp = dp_feature_func(alert_dp)

dp_col = ['cust_id','session_amt_diff_ratio', 
          #,'txbranch_session_cnt','session_atm_txn_ratio','session_cross_bank_ratio',
          'date_time_amt_diff_ratio','tx_cnt_date_time','txbranch_day_time_cnt', 'day_time_atm_txn_ratio','day_time_cross_bank_ratio',
          'date_amt_diff_ratio','tx_cnt_date','txbranch_day_cnt','day_atm_txn_ratio','day_cross_bank_ratio',#(version : original)
          'time_diff',#(version : date_diff_30)
          'tx_cnt_session',#(version : tx_cnt_session)
          #'prev7d_amt_diff_ratio',
          #'prev_7d_tx_cnt','prev_7d_txbranch_ratio',#'prev_7d_atm_cnt','prev_7d_atm_ratio',
          #'prev_7d_txbranch_cnt',#'prev_7d_cross_bank_sum',#'prev_7d_cross_bank_ratio',
          'y']
dp_X, dp_model_1, dp_feature_importance = model_training_1(train_dp, dp_col)
dp_result = result_preprocess_func(train_dp, dp_X, dp_model_1)
dp_model_2, dp_result_col = model_training_2(dp_result)
dp_col.remove('y')
dp_col.remove('cust_id')
dp_final, alert_dp_result = alert_output(alert_dp, dp_model_1, dp_model_2, dp_col, dp_result_col, doc)
dp_final
#F1 Scoure: 66.66%
#0.9980544747081712
#0.5003657644476956
#F1 Scoure: 28.57%
#0.875
#0.17073170731707318

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Accuracy: 92.72%
F1 Scoure: 69.88%
0.9897229669347631
0.54011216776396


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Accuracy: 96.99%
F1 Scoure: 32.65%
1.0
0.1951219512195122


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,alert_key,probability
0,357307,0.034584
1,376329,0.000000
2,373644,0.000000
3,357668,0.040039
4,354443,0.000000
...,...,...
3845,364485,0.032146
3846,363155,0.000000
3847,368710,0.000000
3848,358067,0.000000


In [34]:
dp_feature_importance

,columns,score
0,date_time_amt_diff_ratio,0.044453
1,tx_cnt_date_time,0.051006
2,txbranch_day_time_cnt,0.010780
3,day_time_atm_txn_ratio,0.000000
4,day_time_cross_bank_ratio,0.057026
5,date_amt_diff_ratio,0.000000
6,tx_cnt_date,0.461015
7,txbranch_day_cnt,0.027336
8,day_atm_txn_ratio,0.071687
9,day_cross_bank_ratio,0.050062


In [35]:
dp_final.to_csv('/content/drive/MyDrive/訓練資料集_first/submission.csv')

In [ ]:
###下方為測試區###

In [ ]:
#cdtx zone
train_cdtx, alert_cdtx = train_alert_process_func(cdtx, custinfo, predict_alert_time)
train_cdtx = cdtx_feature_func(train_cdtx)
alert_cdtx = cdtx_feature_func(alert_cdtx)
cdtx_col = ['cust_id', 'amt', 'all_amt_sum','date_amt_sum', 'cdtx_all_txn_cnt', 'date_txn_cnt',
       'cdtx_all_country_cnt', 'date_country_cnt', 'cdtx_all_cur_type_cnt',
       'date_cur_type_cnt', 'max_min_date_diff', 'y']
cdtx_X, cdtx_model_1, cdtx_feature_importance = model_training_1(train_cdtx, cdtx_col)
cdtx_result = result_preprocess_func(train_cdtx, cdtx_X, cdtx_model_1)
cdtx_model_2, cdtx_result_col = model_training_2(cdtx_result)
cdtx_col.remove('y')
cdtx_col.remove('cust_id')
cdtx_final, alert_cdtx_result = alert_output(alert_cdtx, cdtx_model_1, cdtx_model_2, cdtx_col, cdtx_result_col, doc)
cdtx_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Accuracy: 97.98%
F1 Scoure: 94.63%
0.9999391394315623
0.898182315156485


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Accuracy: 96.88%
F1 Scoure: 31.25%
0.625
0.20833333333333334


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,alert_key,probability
0,357307,0.343419
1,376329,0.000000
2,373644,0.000000
3,357668,0.018519
4,354443,0.000000
...,...,...
3845,364485,0.018519
3846,363155,0.000000
3847,368710,0.000000
3848,358067,0.000000


In [ ]:
#remit zone
train_remit, alert_remit = train_alert_process_func(remit, custinfo, predict_alert_time)
train_remit = remit_feature_func(train_remit)
alert_remit = remit_feature_func(alert_remit)
remit_col = ['cust_id','trade_amount_usd','time_amt_sum','all_amt_sum','max_min_date_diff','date_txn_cnt',#,'all_txn_cnt'
                   'remit_trans_no_0','y']
remit_X, remit_model_1, remit_feature_importance = model_training_1(train_remit, remit_col)
remit_result = result_preprocess_func(train_remit, remit_X, remit_model_1)
remit_model_2, remit_result_col = model_training_2(remit_result)
remit_col.remove('y')
remit_col.remove('cust_id')
remit_final, alert_remit_result = alert_output(alert_remit, remit_model_1, remit_model_2, remit_col, remit_result_col, doc)
remit_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Accuracy: 96.23%
F1 Scoure: 95.31%
0.9672466734902764
0.9393638170974155
Accuracy: 94.76%
F1 Scoure: 70.59%
0.8571428571428571
0.6


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,alert_key,probability
0,357307,0.000000
1,376329,0.000000
2,373644,0.000000
3,357668,0.008409
4,354443,0.000000
...,...,...
3845,364485,0.000000
3846,363155,0.000000
3847,368710,0.000000
3848,358067,0.000000


In [ ]:
dp_result['dp_probability'] = dp_model_2.predict_proba(dp_result[dp_result.columns[1:-1]])[:,1]
dp_result = dp_result[['cust_id','dp_probability','y']]
remit_result['remit_probability'] = remit_model_2.predict_proba(remit_result[remit_result.columns[1:-1]])[:,1]
remit_result = remit_result[['cust_id','remit_probability','y']]
cdtx_result['cdtx_probability'] = cdtx_model_2.predict_proba(cdtx_result[cdtx_result.columns[1:-1]])[:,1]
cdtx_result = cdtx_result[['cust_id','cdtx_probability','y']]

In [ ]:
merged_result = dp_result.merge(remit_result, on='cust_id', how='outer')
merged_result = merged_result.merge(cdtx_result, on='cust_id', how='outer')
merged_result['y_x'].fillna(merged_result['y_y'], inplace=True)
merged_result['y_x'].fillna(merged_result['y'], inplace=True)
merged_result = merged_result[['cust_id', 'dp_probability', 'remit_probability', 'cdtx_probability', 'y_x']]
merged_result.rename(columns={'y_x':'y'}, inplace=True)
merged_result.fillna(0, inplace=True)
merged_result = merged_result.merge(train_dp.groupby('cust_id')['tx_amt'].count().rename('dp_tx_cnt').reset_index(), on='cust_id', how='left')
merged_result = merged_result.merge(train_remit.groupby('cust_id')['trade_amount_usd'].count().rename('remit_tx_cnt').reset_index(), on='cust_id', how='left')
merged_result = merged_result.merge(train_cdtx.groupby('cust_id')['amt'].count().rename('cdtx_tx_cnt').reset_index(), on='cust_id', how='left')
merged_result = merged_result[['cust_id', 'dp_probability', 'remit_probability', 'cdtx_probability','dp_tx_cnt','remit_tx_cnt','cdtx_tx_cnt','y']]
merged_result.fillna(0, inplace=True)
merged_result

,cust_id,dp_probability,remit_probability,cdtx_probability,dp_tx_cnt,remit_tx_cnt,cdtx_tx_cnt,y
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,0.029684,0.000000,0.018519,10.0,0.0,84.0,0.0
1,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,0.024050,0.000000,0.018519,1.0,0.0,362.0,0.0
2,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,0.013518,0.008409,0.018519,4.0,1.0,206.0,0.0
3,0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...,0.061374,0.000000,0.018519,28.0,0.0,614.0,0.0
4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,0.113276,0.008409,0.018519,105.0,5.0,3.0,0.0
...,...,...,...,...,...,...,...,...
5698,faf174997b7a4419393c07a39ad65b769bca03bbe0a75c...,0.000000,0.000000,0.018519,0.0,0.0,80.0,0.0
5699,fc0c584f7d83aea221ffe5e8612db2ffc4aa0660bc7c9d...,0.000000,0.000000,0.018519,0.0,0.0,404.0,0.0
5700,fc0fd935773c40ff7719e56d67da572b5c6ffd41361425...,0.000000,0.000000,0.018519,0.0,0.0,18.0,0.0
5701,fca86cee0b3f8b6ec8c4a29d9a7e4798d5de459bdea84d...,0.000000,0.000000,0.018519,0.0,0.0,347.0,0.0


In [ ]:
#dp score
#0.875
#0.17073170731707318
merged_model_2, merged_result_col = model_training_2(merged_result)

Accuracy: 97.20%
F1 Scoure: 61.90%
0.7428571428571429
0.5306122448979592


In [ ]:
alert_cdtx_result.rename(columns={'probability':'cdtx_probability'}, inplace=True)
alert_remit_result.rename(columns={'probability':'remit_probability'}, inplace=True)
alert_dp_result.rename(columns={'probability':'dp_probability'}, inplace=True)
merged_result = alert_dp_result.merge(alert_remit_result, on='cust_id', how='outer')
merged_result = merged_result.merge(alert_cdtx_result, on='cust_id', how='outer')
merged_result = merged_result[['cust_id', 'dp_probability', 'remit_probability', 'cdtx_probability']]
merged_result.fillna(0, inplace=True)
merged_result = merged_result.merge(alert_dp.groupby('cust_id')['tx_amt'].count().rename('dp_tx_cnt').reset_index(), on='cust_id', how='left')
merged_result = merged_result.merge(alert_remit.groupby('cust_id')['trade_amount_usd'].count().rename('remit_tx_cnt').reset_index(), on='cust_id', how='left')
merged_result = merged_result.merge(alert_cdtx.groupby('cust_id')['amt'].count().rename('cdtx_tx_cnt').reset_index(), on='cust_id', how='left')
merged_result.fillna(0, inplace=True)
merged_result['cnt_sum'] = merged_result['dp_tx_cnt'] + merged_result['remit_tx_cnt'] + merged_result['cdtx_tx_cnt']
merged_result['dp_tx_cnt'] = merged_result['dp_tx_cnt']/merged_result['cnt_sum']
merged_result['remit_tx_cnt'] = merged_result['remit_tx_cnt']/merged_result['cnt_sum']
merged_result['cdtx_tx_cnt'] = merged_result['cdtx_tx_cnt']/merged_result['cnt_sum']
merged_result.fillna(0, inplace=True)
merged_result

,cust_id,dp_probability,remit_probability,cdtx_probability,dp_tx_cnt,remit_tx_cnt,cdtx_tx_cnt
0,0172056578071e83399216fcd640bdc4de4583149d0fd9...,0.024050,0.000000,0.000000,23.0,0.0,0.0
1,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,0.024050,0.000000,0.018519,18.0,0.0,183.0
2,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,0.051820,0.033257,0.000000,15.0,6.0,0.0
3,01c705b457297805c1c08137cb3adb164b8f393c265db2...,0.109747,0.000000,0.018519,11059.0,0.0,146.0
4,022bec0a12b56285bf12c49ba1512c0acdbf4176a30d96...,0.022801,0.000000,0.000000,8.0,0.0,0.0
...,...,...,...,...,...,...,...
716,7309609845fb52332d716a6226261ad14d31f2785b5568...,0.000000,0.000000,0.940219,0.0,0.0,1069.0
717,8a04784e4c615c8e8514c0974c1930c1db2816548d5e74...,0.000000,0.000000,0.018519,0.0,0.0,39.0
718,9177ddc696c399ca15bc720866979200c060fe6b5bf7a7...,0.000000,0.000000,0.018519,0.0,0.0,3.0
719,b5c5c235e898032b0dd184993f250ea6bd743c521dc2a5...,0.000000,0.000000,0.940219,0.0,0.0,1660.0


In [ ]:
merged_result_pred = merged_model_2.predict_proba(merged_result[['dp_probability','remit_probability','cdtx_probability','dp_tx_cnt','remit_tx_cnt','cdtx_tx_cnt']])
# evaluate predictions
merged_alert_result = merged_result[['cust_id']]
merged_alert_result['probability'] = merged_result_pred[:,1]

final = predict_alert_time.merge(custinfo[['alert_key', 'cust_id']].merge(merged_alert_result, on='cust_id'), on='alert_key', how='left')[['alert_key', 'probability']]
doc = doc[['alert_key']]
final = doc.merge(final, on='alert_key', how='left')
final.fillna(0,inplace=True)
final.loc[final[final['alert_key'].isin(prev_list)].index,'probability']=0
#mix 
merged_final = final.merge(remit_final, on='alert_key')
merged_final_col = list(merged_final.columns)
merged_final_col.remove('alert_key')
merged_final['probability'] = merged_final[merged_final_col].max(axis=1)
merged_final = merged_final[['alert_key', 'probability']]
merged_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
  #交易金額重複次數
  #max_tx_amt_cnt = data.groupby(['cust_id'])['tx_amt'].value_counts().groupby('cust_id').max().rename('max_tx_amt_cnt')
  #data = data.merge(max_tx_amt_cnt, on=['cust_id'], how='left')
  #tx_amt_cnt = data.groupby(['cust_id'])['tx_amt'].value_counts().rename('all_tx_amt_cnt').reset_index()
  #data = data.merge(tx_amt_cnt, on=['cust_id','tx_amt'], how='left')
  #當日交易金額重複次數
  #date_max_tx_amt_cnt = data.groupby(['cust_id','tx_date'])['tx_amt'].value_counts().groupby(['cust_id','tx_date']).max().rename('date_max_tx_amt_cnt')
  #data = data.merge(date_max_tx_amt_cnt, on=['cust_id','tx_date'], how='left')

  #tx_amt_cnt = data.groupby(['cust_id','tx_date'])['tx_amt'].value_counts().rename('date_tx_amt_cnt').reset_index()
  #data = data.merge(tx_amt_cnt, on=['cust_id','tx_date','tx_amt'], how='left')
  #當sessionATM 佔交易數比例
  #session_atm_txn_ratio = data.groupby(['session_cust_id'])['ATM'].sum().reset_index()
  #session_atm_txn_ratio.rename(columns={'ATM':'session_atm_txn_ratio'}, inplace=True)
  #data = data.merge(session_atm_txn_ratio, on=['session_cust_id'], how='left')
  #data.session_atm_txn_ratio = data.session_atm_txn_ratio / data.tx_cnt_date
  #當session跨行 佔交易數比例
  #session_cross_bank_ratio = data.groupby(['session_cust_id'])['cross_bank'].sum().reset_index()
  #session_cross_bank_ratio.rename(columns={'cross_bank':'session_cross_bank_ratio'}, inplace=True)
  #data = data.merge(session_cross_bank_ratio, on=['session_cust_id'], how='left')
  #data.session_cross_bank_ratio = data.session_cross_bank_ratio / data.tx_cnt_date
  #當session交易筆數 tx_cnt_session
  #tx_cnt_session = data.groupby(['session_cust_id'])['debit_credit'].count().reset_index()
  #tx_cnt_session.rename(columns={'debit_credit':'tx_cnt_session'}, inplace=True)
  #data = data.merge(tx_cnt_session, on=['session_cust_id'], how='left')
#當session總分行數 txbranch_session_cnt
  #txbranch_session_cnt = data.groupby(['session_cust_id'])['txbranch'].count().reset_index()
  #txbranch_session_cnt.rename(columns={'txbranch':'txbranch_session_cnt'}, inplace=True)
  #data = data.merge(txbranch_session_cnt, on=['session_cust_id'], how='left')